In [1]:
%pip install gdal

In [2]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
hdf_file

In [4]:
import gdal
hdf_file = gdal.Open("/content/drive/MyDrive/6962 final project/data/Vegetation/MOD13C1.A2000097.061.2020041164519.hdf") 

subDatasets = hdf_file.GetSubDatasets()

# Open precipitation
# prcp = gdal.Open('HDF4_SDS:UNKNOWN:"3B43.20140501.7.HDF":0')
# or the following shortcut:
temp = gdal.Open(subDatasets[0][0])
temp = temp.ReadAsArray()
temp = pd.DataFrame(temp)

In [5]:
def read_hdf(path):
  return pd.DataFrame(gdal.Open((gdal.Open(path).GetSubDatasets())[0][0]).ReadAsArray())        

In [6]:
subDatasets

[('HDF4_EOS:EOS_GRID:"/content/drive/MyDrive/6962 final project/data/Vegetation/MOD13C1.A2000097.061.2020041164519.hdf":MODIS_Grid_16Day_VI_CMG:CMG 0.05 Deg 16 days NDVI',
  '[3600x7200] CMG 0.05 Deg 16 days NDVI MODIS_Grid_16Day_VI_CMG (16-bit integer)'),
 ('HDF4_EOS:EOS_GRID:"/content/drive/MyDrive/6962 final project/data/Vegetation/MOD13C1.A2000097.061.2020041164519.hdf":MODIS_Grid_16Day_VI_CMG:CMG 0.05 Deg 16 days EVI',
  '[3600x7200] CMG 0.05 Deg 16 days EVI MODIS_Grid_16Day_VI_CMG (16-bit integer)'),
 ('HDF4_EOS:EOS_GRID:"/content/drive/MyDrive/6962 final project/data/Vegetation/MOD13C1.A2000097.061.2020041164519.hdf":MODIS_Grid_16Day_VI_CMG:CMG 0.05 Deg 16 days VI Quality',
  '[3600x7200] CMG 0.05 Deg 16 days VI Quality MODIS_Grid_16Day_VI_CMG (16-bit unsigned integer)'),
 ('HDF4_EOS:EOS_GRID:"/content/drive/MyDrive/6962 final project/data/Vegetation/MOD13C1.A2000097.061.2020041164519.hdf":MODIS_Grid_16Day_VI_CMG:CMG 0.05 Deg 16 days red reflectance',
  '[3600x7200] CMG 0.05 Deg

In [7]:
temp

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,7160,7161,7162,7163,7164,7165,7166,7167,7168,7169,7170,7171,7172,7173,7174,7175,7176,7177,7178,7179,7180,7181,7182,7183,7184,7185,7186,7187,7188,7189,7190,7191,7192,7193,7194,7195,7196,7197,7198,7199
0,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,...,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000
1,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,...,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000
2,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,...,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000
3,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,...,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000
4,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,...,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,...,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000
3596,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,...,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-30

In [ ]:
#temp[3600][1800]
temp[4800][2500]

-3000

#Matrix Transformation

In [8]:
df_test = temp.copy()

In [9]:
# df_test.index = (df_test.index/3600*-180) +90
# df_test.columns =  (360/7200)*df_test.columns- 180

In [10]:
df_out = pd.DataFrame(columns=['veg'])
df_out

,veg


In [11]:
veg_list = [ pd.DataFrame(data= np.array(i[1]) ,columns=['veg']) for i in df_test.iterrows()]

In [12]:
df_out = pd.concat(veg_list)

In [13]:
lat_list = list(range(0,3600)) * 7200
lat_list.sort()
lon_list = list(range(0,7200)) * 3600

In [14]:
df_out['lat'] = lat_list
df_out['lon'] = lon_list

In [15]:
df_out['lat'] = np.round((df_out['lat']/3600*-180) +90,2)
df_out['lon'] = np.round((360/7200)*df_out['lon']- 180,2)

In [16]:
df_out.reset_index(inplace=True,drop=True)

In [ ]:
df_out

,veg,lat,lon
0,-3000,90.00,-180.00
1,-3000,90.00,-179.95
2,-3000,90.00,-179.90
3,-3000,90.00,-179.85
4,-3000,90.00,-179.80
...,...,...,...
25919995,-3000,-89.95,179.75
25919996,-3000,-89.95,179.80
25919997,-3000,-89.95,179.85
25919998,-3000,-89.95,179.90


In [26]:
df_na = df_out.copy()
df_na = df_na[df_na['veg'] == -3000]

In [27]:
len(df_na)/len(df_out)

0.7577374614197531

#The Function

In [28]:
def matrix_transform(df__):  
  df__ = df__.copy()
  # df_.index = (df_.index/3600*-180) +90
  # df_.columns =  (360/7200)*df_.columns- 180
  df_out_ = pd.DataFrame(columns=['veg'])
  veg_list = [ pd.DataFrame(data= np.array(i[1]) ,columns=['veg']) for i in df__.iterrows()]
  df_out_ = pd.concat(veg_list)
  lat_list = list(range(0,3600)) * 7200
  lat_list.sort()
  lon_list = list(range(0,7200)) * 3600
  df_out_['lat'] = lat_list
  df_out_['lon'] = lon_list
  df_out_['lat'] = np.round((df_out_['lat']/3600*-180) +90,2)
  df_out_['lon'] = np.round((360/7200)*df_out_['lon']- 180,2)
  df_out_.reset_index(inplace=True,drop=True)
  return df_out_

In [29]:
hdf_file = gdal.Open("/content/drive/MyDrive/6962 final project/data/Vegetation/vegeINfo/20001.hdf") 

subDatasets = hdf_file.GetSubDatasets()

# Open precipitation
# prcp = gdal.Open('HDF4_SDS:UNKNOWN:"3B43.20140501.7.HDF":0')
# or the following shortcut:
temp = gdal.Open(subDatasets[0][0])
temp = temp.ReadAsArray()
temp = pd.DataFrame(temp)

In [30]:
df_20001 = matrix_transform(temp)

In [ ]:
list((df_20001['veg'] == -3000).index) == list((df_out['veg'] == -3000).index)

True

In [ ]:
df_test = df_20001.copy()

In [ ]:
df_test['lon'] = (np.floor(df_test['lon'])).astype(int)
df_test['lat'] = (np.floor(df_test['lat'])).astype(int)

In [ ]:
df_group = df_test.groupby(['lat','lon']).agg('mean')

In [ ]:
df_group

veg
lat lon         
-90 -180 -3000.0
    -179 -3000.0
    -178 -3000.0
    -177 -3000.0
    -176 -3000.0
...          ...
 90  175 -3000.0
     176 -3000.0
     177 -3000.0
     178 -3000.0
     179 -3000.0

[65160 rows x 1 columns]

In [ ]:
list(df_group.index)

In [ ]:
df_final_csv = pd.read_csv('/content/drive/MyDrive/6962 final project/data/final_csv.csv')

In [ ]:
df_final_csv['cor'] = df_final_csv.apply(lambda x: (x['lat'],x['lon']),axis=1)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def select_df_needed(df___):
  df_group_no_na = df___.copy()
  df_group_no_na = df_group_no_na.loc[df_group_no_na.veg != -3000]
  df_group_no_na['lat'] = np.floor(df_group_no_na['lat']).astype(int)
  df_group_no_na['lon'] = np.floor(df_group_no_na['lon']).astype(int)
  df_group_no_na = df_group_no_na.groupby(['lat','lon']).agg('mean')
  df_needed = pd.DataFrame()
  for i in list(set(df_final_csv['cor'])):
    try:
      df_needed = df_needed.append(df_group_no_na.loc[i])
    except:
      print(i)
  return df_needed

In [ ]:
import datetime as dt
from datetime import timedelta

In [ ]:
df_final_csv['time'] = pd.to_datetime(df_final_csv['time'])

In [ ]:
min(df_final_csv['time']), max(df_final_csv['time'])

(Timestamp('2000-03-01 00:00:00'), Timestamp('2019-12-31 00:00:00'))

In [ ]:
a = select_df_needed(df_20001)

In [ ]:
dt.datetime(2019,12,31) - dt.datetime(2000,3,1)

datetime.timedelta(days=7244)

In [ ]:
date_list = [Timestamp('2019-12-31 00:00:00') - timedelta(days=x) for x in range(numdays)]

In [ ]:
first_half = select_df_needed(matrix_transform(read_hdf('/content/drive/MyDrive/6962 final project/data/Vegetation/vegeINfo/20001.hdf')))
second_half = select_df_needed(matrix_transform(read_hdf('/content/drive/MyDrive/6962 final project/data/Vegetation/vegeINfo/20002.hdf')))

In [ ]:
first_half

,veg
"(32, 54)",660.042500
"(34, 73)",3271.562500
"(42, 25)",5139.447500
"(43, 20)",4677.595000
"(24, 66)",1297.578947
...,...
"(-30, 22)",3039.685000
"(14, -17)",2012.624339
"(43, 47)",1415.725441
"(23, 70)",1195.080000


In [ ]:
df_veg_out

,veg,year,half
0,690.260000,2015,2
1,4642.672500,2015,2
2,6329.885000,2015,2
3,6634.422500,2015,2
4,1737.380952,2015,2
...,...,...,...
5355,2250.195000,2018,2
5356,2309.232804,2018,2
5357,2941.465753,2018,2
5358,1660.902564,2018,2


In [ ]:
import os
directory = '/content/drive/MyDrive/6962 final project/data/Vegetation/vegeINfo/'

df_veg_list = list()
for filename in os.listdir(directory):
    if filename.endswith(".hdf"):
      print(filename)
      local_df__ = select_df_needed(matrix_transform(read_hdf(directory+filename)))
      local_df__['year'] = np.array([filename[0:4]]*134)
      local_df__['half'] = np.array([filename[4:5]]*134)
      df_veg_list.append(local_df__)
    else:
      continue

new_df_veg_list = [i.reset_index() for i in df_veg_list]
new_df_veg_list[0]      
df_veg_out = pd.concat(new_df_veg_list)
df_veg_out.reset_index(inplace=True, drop=True)
df_veg_out.to_csv('df_veg_out.csv')
print("Finished")

In [ ]:
new_df_veg_list = [i.reset_index() for i in df_veg_list]    
df_veg_out = pd.concat(new_df_veg_list)
df_veg_out.reset_index(inplace=True, drop=True)
df_veg_out.to_csv('df_veg_out.csv')

In [ ]:
df_veg_out

,index,veg,year,half
0,"(32, 54)",690.260000,2015,2
1,"(34, 73)",4642.672500,2015,2
2,"(42, 25)",6329.885000,2015,2
3,"(43, 20)",6634.422500,2015,2
4,"(24, 66)",1737.380952,2015,2
...,...,...,...,...
5355,"(-30, 22)",2250.195000,2018,2
5356,"(14, -17)",2309.232804,2018,2
5357,"(43, 47)",2941.465753,2018,2
5358,"(23, 70)",1660.902564,2018,2


In [ ]:
%cd ..

/content/drive/.shortcut-targets-by-id/12pjah520zwJMQos4SHRxl7lFoUlEuU7h/6962 final project/data/Vegetation


In [ ]:
pd.read_csv('/content/drive/MyDrive/6962 final project/data/Vegetation/df_veg_out.csv')

,Unnamed: 0,index,veg,year,half
0,0,"(32, 54)",690.260000,2015,2
1,1,"(34, 73)",4642.672500,2015,2
2,2,"(42, 25)",6329.885000,2015,2
3,3,"(43, 20)",6634.422500,2015,2
4,4,"(24, 66)",1737.380952,2015,2
...,...,...,...,...,...
5355,5355,"(-30, 22)",2250.195000,2018,2
5356,5356,"(14, -17)",2309.232804,2018,2
5357,5357,"(43, 47)",2941.465753,2018,2
5358,5358,"(23, 70)",1660.902564,2018,2


In [ ]:
df_precip = pd.read_csv('/content/drive/MyDrive/6962 final project/data/final_csv.csv')

In [ ]:
# df_the_actual_out = pd.DataFrame()
# half_year_veg_list = list()
# local_df = df_needed.copy()
# for i in [dt.datetime(2000,3,1) + timedelta(days=i) for i in range(0,7245)]:
#   if i.month > 6:
#     local_df['time'] = np.array([i]*134)
#     half_year_veg_list.append(local_df)


In [ ]:
df_precip['time'] = pd.to_datetime(df_precip['time'])
df_precip['year'] = df_precip.apply(lambda x: x['time'].year,axis = 1)
df_precip['half'] = df_precip.apply(lambda x: 2 if x['time'].month>6 else 1,axis = 1)
df_precip['cor'] = df_precip.apply(lambda x: (x['lat'],x['lon']),axis = 1)

In [ ]:
def veg_merge_precip(x):
  lg_1 = df_veg_out['index'] == x['cor']
  lg_2 = df_veg_out['year'] == x['year']
  lg_3 = df_veg_out['half'] == x['half']
  return (df_veg_out.loc[np.logical_and(np.logical_and(lg_1,lg_2),lg_3)]).veg

In [ ]:
df_precip['veg'] = df_precip.apply(lambda x: veg_merge_precip(x),axis = 1)

In [ ]:
df_veg_out

,index,veg,year,half
0,"(32, 54)",690.260000,2015,2
1,"(34, 73)",4642.672500,2015,2
2,"(42, 25)",6329.885000,2015,2
3,"(43, 20)",6634.422500,2015,2
4,"(24, 66)",1737.380952,2015,2
...,...,...,...,...
5355,"(-30, 22)",2250.195000,2018,2
5356,"(14, -17)",2309.232804,2018,2
5357,"(43, 47)",2941.465753,2018,2
5358,"(23, 70)",1660.902564,2018,2


In [ ]:
a = df_precip.copy()

In [ ]:
sum(df_needed.veg <-2000)

0

In [ ]:
df_needed

,veg
"(32, 54)",660.042500
"(34, 73)",3271.562500
"(42, 25)",5139.447500
"(43, 20)",4677.595000
"(24, 66)",1297.578947
...,...
"(-30, 22)",3039.685000
"(14, -17)",2012.624339
"(43, 47)",1415.725441
"(23, 70)",1195.080000


#Testing

In [ ]:
df_out[df_out['veg'] != -3000]

,veg,lat,lon
910122,-291,83.70,-33.90
910123,-375,83.70,-33.85
910124,-476,83.70,-33.80
910125,-489,83.70,-33.75
910126,-489,83.70,-33.70
...,...,...,...
21516658,348,-59.40,-27.10
21523853,-474,-59.45,-27.35
21523854,-449,-59.45,-27.30
21523855,-283,-59.45,-27.25


In [ ]:
df_test.index = np.round((df_test.index/3600*-180) +90,2)
df_test.columns =  np.round((360/7200)*df_test.columns- 180,2)

In [ ]:
df_test.loc[-59.45,-27.30]

-449

In [ ]:
a = read_hdf('/content/drive/MyDrive/6962 final project/data/Vegetation/vegeINfo/20022.hdf')

In [ ]:
b = read_hdf('/content/drive/MyDrive/6962 final project/data/Vegetation/vegeINfo/20191.hdf')

In [ ]:
a

In [ ]:
b

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,7160,7161,7162,7163,7164,7165,7166,7167,7168,7169,7170,7171,7172,7173,7174,7175,7176,7177,7178,7179,7180,7181,7182,7183,7184,7185,7186,7187,7188,7189,7190,7191,7192,7193,7194,7195,7196,7197,7198,7199
0,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,...,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000
1,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,...,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000
2,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,...,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000
3,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,...,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000
4,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,...,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,...,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000
3596,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,...,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-3000,-30

In [ ]:
def areSame(A,B):
   for i in range(3600):
      for j in range(7200):
         if (A[i][j] != B[i][j]):
            print(i,j)
            return 0
   return 1

In [ ]:
areSame(a,b)

#Deprecated

In [ ]:
def convert_matrix(row,col):
  lon = (360/7200)*col- 180
  lat = (row/3600*-180) +90
  return lat, lon


convert_matrix(1800,1800)[1]

-90.0

In [ ]:
vege = pd.DataFrame(columns=('lat', 'lon', 'vegetation_val'))
for i in range(3600):
  print(i)
  for j in range(7200):
 
    vege.loc[i*7200+j] = [convert_matrix(i,j)[0], convert_matrix(i,j)[1], temp[j][i]]

